# Ejercicio 03: Cálculo de Relevancia y Ranking de Documentos

Integrantes: Rossy Armendariz, Alejandro Chavez, Javier Quishpe

El objetivo de este ejercicio es calcular analíticamente la relevancia de cada documento en un corpus y luego ordenar (rankear) los documentos basándonos en esa puntuación de relevancia para tres consultas específicas. Seguirás los siguientes pasos:

Descripción del Ejercicio

1. Procesamiento del Corpus:
    * Leer y parsear el archivo XML proporcionado que contiene el corpus de documentos con sus metadatos (keywords, autor y fecha).

2. Procesamiento de las Consultas:
    * Definir las consultas proporcionadas.
    * Extraer las palabras clave de cada consulta.

3. Cálculo de Relevancia:
    * Utilizar métricas de similitud (Similitud Coseno y Jaccard) entre la representación vectorial de los documentos y las de las consultas.
    * Calcular la puntuación de relevancia para cada documento del corpus respecto a cada consulta.

4. Ranking de Documentos:
    * Ordenar los documentos en función de su puntuación de relevancia de mayor a menor.
    * Mostrar el ranking de documentos para cada consulta.

In [15]:
# Importar las librerías necesarias
import xml.etree.ElementTree as ET

# Función para procesar el texto y extraer palabras clave
def process_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Reemplazar caracteres no alfanuméricos por espacios
    import re
    text = re.sub(r'[^a-záéíóúñü]+', ' ', text)
    # Tokenizar y eliminar palabras vacías si es necesario
    tokens = text.strip().split()
    return set(tokens)

# Paso 1: Leer y parsear el archivo XML
def parse_corpus(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    corpus = {}
    for doc in root.findall('document'):
        doc_id = int(doc.get('id'))
        title = doc.find('title').text
        keywords = doc.find('keywords').text
        author = doc.find('author').text
        date = doc.find('date').text
        # Procesar las palabras clave
        keyword_set = process_text(keywords)
        corpus[doc_id] = {
            'title': title,
            'keywords': keyword_set,
            'author': author,
            'date': date
        }
    return corpus

In [16]:
corpus = parse_corpus('./data/03ranking_corpus.xml')
corpus

{1: {'title': 'El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas.',
  'keywords': {'crónica',
   'médica',
   'salud',
   'tecnología',
   'telemedicina',
   'tratamiento'},
  'author': 'Dr. Juan Pérez',
  'date': '2023-01-15'},
 2: {'title': 'Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.',
  'keywords': {'académico',
   'estudiantes',
   'mental',
   'nutrición',
   'rendimiento',
   'salud'},
  'author': 'Dra. María López',
  'date': '2023-02-10'},
 3: {'title': 'Estudio sobre cómo las relaciones de amistad contribuyen al bienestar de los estudiantes en el campus.',
  'keywords': {'amistad',
   'bienestar',
   'campus',
   'estudiantil',
   'relaciones',
   'sociales'},
  'author': 'Miguel Rodríguez',
  'date': '2023-03-05'},
 4: {'title': 'El rol de las bibliotecas universitarias en el fomento de la investigación académica.',
  'keywords': {'academia',
   'bibliotecas',
   'investigación',
   'recursos

# Procesamiento de las Consultas

Se define las consultas.

In [17]:
# Definir las consultas
queries = {
    1: "Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios",
    2: "Actividades extracurriculares y bienestar emocional en el campus universitario",
    3: "Estrategias universitarias para reducir el estrés en estudiantes"
}

Se extrae las palabras clave de cada consulta.

In [18]:
for q_id, query in queries.items():
    query_keywords = process_text(query)
    print(f"Consulta {q_id}: {query}")
    print(f"Palabras clave: {query_keywords}\n")

Consulta 1: Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios
Palabras clave: {'la', 'mental', 'impacto', 'rendimiento', 'los', 'en', 'académico', 'de', 'estudiantes', 'universitarios', 'el', 'salud'}

Consulta 2: Actividades extracurriculares y bienestar emocional en el campus universitario
Palabras clave: {'y', 'en', 'emocional', 'el', 'actividades', 'universitario', 'bienestar', 'extracurriculares', 'campus'}

Consulta 3: Estrategias universitarias para reducir el estrés en estudiantes
Palabras clave: {'reducir', 'para', 'estrés', 'en', 'universitarias', 'estrategias', 'estudiantes', 'el'}



# Cálculo de Relevancia

Se utiliza las métricas de similitud Coseno y Jaccard entre la representación vectorial de los documentos y las de las consultas.

In [19]:
import math

# Calcular Similitud de Coseno
def cosine_similarity(set1, set2):
    intersection = set1.intersection(set2)
    return len(intersection) / math.sqrt(len(set1) * len(set2)) if set1 and set2 else 0

# Calcular Similitud de Jaccard
def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union) if union else 0

Se calcula la puntuación de relevancia para cada documento del corpus respecto a cada consulta.

In [20]:
# Diccionario para almacenar las puntuaciones de relevancia para cada consulta
scores = {q_id: [] for q_id in queries}

for q_id, query in queries.items():
    query_keywords = process_text(query)
    for doc_id, doc_data in corpus.items():
        doc_keywords = doc_data['keywords']
        jaccard_score = jaccard_similarity(query_keywords, doc_keywords)
        cosine_score = cosine_similarity(query_keywords, doc_keywords)
        relevance_score = (jaccard_score + cosine_score) / 2 # Promedio para la puntuación de relevancia
        scores[q_id].append((doc_id, relevance_score))

# Ranking de Documentos
Se ordena los documentos por relevancia para cada consulta respectiva.

In [21]:
ranked_docs = {q_id: sorted(scores[q_id], key=lambda x: x[1], reverse=True) for q_id in queries}

Mostrar el ranking de documentos para cada consulta.

In [22]:
for q_id, ranking in ranked_docs.items():
    print(f"Consulta {q_id}: {queries[q_id]}")
    for rank, (doc_id, score) in enumerate(ranking, start=1):
        print(f"  {rank}. Documento ID {doc_id} - Puntuación de relevancia: {score:.4f}")

Consulta 1: Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios
  1. Documento ID 2 - Puntuación de relevancia: 0.4869
  2. Documento ID 7 - Puntuación de relevancia: 0.4869
  3. Documento ID 14 - Puntuación de relevancia: 0.4513
  4. Documento ID 13 - Puntuación de relevancia: 0.3968
  5. Documento ID 11 - Puntuación de relevancia: 0.3786
  6. Documento ID 19 - Puntuación de relevancia: 0.3291
  7. Documento ID 29 - Puntuación de relevancia: 0.3291
  8. Documento ID 18 - Puntuación de relevancia: 0.2574
  9. Documento ID 12 - Puntuación de relevancia: 0.1958
  10. Documento ID 21 - Puntuación de relevancia: 0.1679
  11. Documento ID 23 - Puntuación de relevancia: 0.1679
  12. Documento ID 26 - Puntuación de relevancia: 0.1679
  13. Documento ID 1 - Puntuación de relevancia: 0.0883
  14. Documento ID 6 - Puntuación de relevancia: 0.0883
  15. Documento ID 8 - Puntuación de relevancia: 0.0883
  16. Documento ID 15 - Puntuación de relevancia: 0.0883
 